In [ ]:
import os
from Dataloader import *
import folium
import zipfile
import requests

import pandas as pd
import pyarrow
import pyarrow.parquet
import matplotlib.pyplot as plt

In [ ]:
def download(url, filepath):
    """Simple download with progress tracking"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    downloaded_size = 0
    
    with open(filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                downloaded_size += len(chunk)
                
                # Progress bar
                if total_size > 0:
                    percent = (downloaded_size / total_size) * 100
                    mb_downloaded = downloaded_size / (1024 * 1024)
                    mb_total = total_size / (1024 * 1024)
                    print(f"\r📥 Progress: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')
    
    print()
    print(f"✅ Download complete: {filepath}")

In [ ]:
data_links = {"Jan": "http://aisdata.ais.dk/2023/aisdk-2023-01.zip",
              "Feb": "http://aisdata.ais.dk/2023/aisdk-2023-02.zip",
              "Mar": "http://aisdata.ais.dk/2023/aisdk-2023-03.zip",
              "Apr": "http://aisdata.ais.dk/2023/aisdk-2023-04.zip",
              "May": "http://aisdata.ais.dk/2023/aisdk-2023-05.zip",
              "Jun": "http://aisdata.ais.dk/2023/aisdk-2023-06.zip",
              "Jul": "http://aisdata.ais.dk/2023/aisdk-2023-07.zip",
              "Aug": "http://aisdata.ais.dk/2023/aisdk-2023-08.zip",
              "Sep": "http://aisdata.ais.dk/2023/aisdk-2023-09.zip",
              "Oct": "http://aisdata.ais.dk/2023/aisdk-2023-10.zip",
              "Nov": "http://aisdata.ais.dk/2023/aisdk-2023-11.zip",
              "Dec": "http://aisdata.ais.dk/2023/aisdk-2023-12.zip"}

data_dir = "../data/unprocessed_data"
end_dir = "../data/processed_data"
os.makedirs(data_dir, exist_ok=True)
os.makedirs(end_dir, exist_ok=True)

for month, link in data_links.items():
    filename = link.split("/")[-1]
    filepath = os.path.join(data_dir, filename)

    # Extract month number from filename
    month_num = filename.split("-")[2].replace(".zip", "")

    # Check if all parquet files for this month already exist
    # Quick check: if folder has files matching this month pattern
    existing_parquets = [f for f in os.listdir(end_dir) if f.startswith(f"aisdk-2023-{month_num}-") and f.endswith('.parquet')]
    
    # A month should have 28-31 parquet files
    if len(existing_parquets) >= 28:
        print(f"⏭️  Skipping month {month}: {len(existing_parquets)} parquet files already exist")
        continue

    # Download if needed
    if os.path.exists(filepath):
        print(f"⏭️  Skipping download for month {month}: {filepath} (already exists)")
    else:
        print(f"Downloading data for {month}...")
        download(link, filepath)
        print(f"Downloaded data for {month}")

    # Open ZIP once and process all CSV files
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
        print(f"Found {len(csv_files)} CSV files in {filename}")
        
        for csv_filename in csv_files:
            output_filename = csv_filename.replace('.csv', '.parquet')
            output_path = os.path.join(end_dir, output_filename)
            
            if os.path.exists(output_path):
                print(f"⏭️  Skipping {output_filename} (already exists)")
                continue
            
            print(f"📝 Processing: {csv_filename}")
            
            data = Dataloader(
                file_path="",
                out_path=output_path,
                zip_path=filepath,
                csv_internal_path=csv_filename
            )
            data.clean_data()

    print(f"✅ Processed {month}")

    # Remove ZIP after processing
    os.remove(filepath)
    print(f"🗑️  Removed {filename}")

print("🎉 All data processed")

# Plotting the clean data

In [ ]:
import pandas as pd
import pyarrow
import pyarrow.parquet
from Dataloader import *
import matplotlib.pyplot as plt
import folium
import os


In [ ]:
path = "../data/processed_data"
dataloader = Dataloader(out_path=path)
# df = dataloader.load_data(date_folders = ["aisdk-2023-01-01.parquet"]) # for specific files
df = dataloader.load_data()  # load all files in the processed_data folder
# Ensure ship and segment can be told apart by adding column for date
df['Date'] = df['Timestamp'].dt.date

In [ ]:
# We are encountering that a ships sailing over midnight gets split into two segments, even though it's the same journey.
# We create a function to overhaul the segment IDs accordingly.
# This is happening because in the data cleaning function each day is handled separately, causing segments to reset at midnight.
def overhaul_segments(df, time_threshold_minutes=7.5):
    """
    Overhauls segment IDs based on a time threshold between consecutive points.
    A new segment is created for a ship if the time gap between two of its
    data points is greater than the specified threshold.

    input:
        df: DataFrame with 'MMSI' and 'Timestamp' columns.
        time_threshold_minutes: The maximum time gap in minutes before a new segment is started.
    output:
        df: DataFrame with a new 'Segment_ID' column.
    """
    # Data is sorted correctly
    df = df.sort_values(by=['MMSI', 'Timestamp']).reset_index(drop=True)

    # Calculate the time difference between consecutive points for each ship
    time_diffs = df.groupby('MMSI')['Timestamp'].diff()

    # A new segment starts if the time difference is larger than our threshold
    # or if it's the first point for a new ship (where time_diff is NaT)
    new_segment_starts = (time_diffs > pd.Timedelta(minutes=time_threshold_minutes)) | (time_diffs.isna())

    # Use cumsum() to create a unique, incrementing ID for each segment
    df['Segment_ID'] = new_segment_starts.cumsum()

    # To make segment IDs restart from 0 for each ship, we can do:
    df['Segment_ID'] = df.groupby('MMSI')['Segment_ID'].transform(lambda x: x - x.min())
    
    return df

df = overhaul_segments(df)
df.drop(columns=['Segment'], inplace=True)
df.rename(columns={"Segment_ID": "Segment"}, inplace=True)



In [ ]:
# Visualize every vessel's trajectory on a map - RESPECTING SEGMENTS
m = folium.Map(location=[55.6761, 12.5683], zoom_start=6)

# Plot each vessel's trajectory, with separate polylines for each segment
for vessel_id in df["MMSI"].unique()[:int(0.5*len(df["MMSI"].unique()))]: # limit to first 50% of vessels
    vessel_data = df[df["MMSI"] == vessel_id]
    
    # ✅ FIXED: Group by BOTH Segment AND Date
    for (segment_id, date), segment in vessel_data.groupby(['Segment', 'Date']):
        # Sort by timestamp within the segment
        segment = segment.sort_values('Timestamp')
        
        # Extract coordinates
        points = list(zip(segment['Latitude'], segment['Longitude']))
        
        # Only plot if we have at least 2 points to make a line
        if len(points) >= 2:
            folium.PolyLine(
                locations=points,
                color="blue",
                weight=2,
                opacity=0.6,
                popup=f"Vessel {vessel_id}<br>Segment {segment_id}<br>{len(points)} points"
            ).add_to(m)
m

In [ ]:
# Visualize every vessel's trajectory on a map - RESPECTING SEGMENTS
m = folium.Map(location=[55.6761, 12.5683], zoom_start=6)

# Plot each vessel's trajectory, with separate polylines for each segment
for vessel_id in df["MMSI"].unique()[int(0.5*len(df["MMSI"].unique())):]: # limit to last 50% of vessels
    vessel_data = df[df["MMSI"] == vessel_id]
    
    # ✅ FIXED: Group by BOTH Segment AND Date
    for (segment_id, date), segment in vessel_data.groupby(['Segment', 'Date']):
        # Sort by timestamp within the segment
        segment = segment.sort_values('Timestamp')
        
        # Extract coordinates
        points = list(zip(segment['Latitude'], segment['Longitude']))
        
        # Only plot if we have at least 2 points to make a line
        if len(points) >= 2:
            folium.PolyLine(
                locations=points,
                color="blue",
                weight=2,
                opacity=0.6,
                popup=f"Vessel {vessel_id}<br>Segment {segment_id}<br>{len(points)} points"
            ).add_to(m)
m